In [20]:
import math
import heapq
import time

N = 200
grid = [[0]*N for _ in range(N)]
# List of obstacle rectangles: (x1, y1, x2, y2)
obstacles = [
    (40, 40, 70, 70),
    (90, 120, 130, 160),
    (150, 50, 180, 90),
    (60, 150, 100, 190)
]
#making the obstacle regions as 1 in the grid
for x1, y1, x2, y2 in obstacles:
    for i in range(x1, x2 + 1):
        for j in range(y1, y2 + 1):
            grid[i][j] = 1



In [21]:
#checking is the node a valid node or not
def isValid(x, y, grid):
    n = len(grid)
    if x < 0 or y < 0 or x >= n or y >= n:
        return False
    if grid[x][y] == 1:
        return False
    return True

In [22]:
def dijkstras(source, grid):
    counter=0
    n = len(grid)
    # making a movements array
    movements = [
        (1, 0, -1),   # left
        (1, 0, 1),    # right
        (1, -1, 0),   # up
        (1, 1, 0),    # down
        (math.sqrt(2), 1, 1),     # down-right
        (math.sqrt(2), -1, 1),    # up-right
        (math.sqrt(2), 1, -1),    # down-left
        (math.sqrt(2), -1, -1)    # up-left
    ]
    distance = [[float('inf')] * n for _ in range(n)]
    pq = []
    sx, sy = source
    distance[sx][sy] = 0
    heapq.heappush(pq, (0, sx, sy))
    while pq:
        currDist, x, y = heapq.heappop(pq)
        if currDist > distance[x][y]:
            continue
        counter+=1
        for cost, dx, dy in movements:
            nx = x + dx
            ny = y + dy
            if not isValid(nx, ny, grid):
                continue
            newDist = currDist + cost
            if newDist < distance[nx][ny]:
                distance[nx][ny] = newDist
                heapq.heappush(pq, (newDist, nx, ny))

    return distance,counter


In [23]:
source = (0, 0)
end = (199, 199)
start_time = time.perf_counter()
dist,expandedNodes = dijkstras(source, grid)
end_time = time.perf_counter()
dijkstrasTime = end_time - start_time
print("Dijkstras distance:", dist[199][199])
print("Dijkstras expanded nodes:", expandedNodes)
print("Dijkstras time :", dijkstrasTime)

Dijkstras distance: 299.5878784786809
Dijkstras expanded nodes: 34527
Dijkstras time : 0.09990268499996091


In [24]:
def heuristic(x, y, gx, gy):
    dx = abs(x - gx)
    dy = abs(y - gy)
    return (max(dx, dy) - min(dx, dy)) + min(dx, dy) * math.sqrt(2)

def astar(source, goal, grid):
    counter = 0
    n = len(grid)
    movements = [
        (1, 0, -1), (1, 0, 1), (1, -1, 0), (1, 1, 0),
        (math.sqrt(2), 1, 1), (math.sqrt(2), -1, 1),
        (math.sqrt(2), 1, -1), (math.sqrt(2), -1, -1)
    ]
    distance = [[float('inf')] * n for _ in range(n)]
    pq = []
    sx, sy = source
    gx, gy = goal
    distance[sx][sy] = 0
    h0 = heuristic(sx, sy, gx, gy)
    heapq.heappush(pq, (h0, 0, sx, sy))
    while pq:
        f, currDist, x, y = heapq.heappop(pq)
        if currDist > distance[x][y]:
            continue
        counter += 1
        if (x, y) == (gx, gy):
            break
        for cost, dx, dy in movements:
            nx = x + dx
            ny = y + dy
            if not isValid(nx, ny, grid):
                continue
            newDist = currDist + cost
            if newDist < distance[nx][ny]:
                distance[nx][ny] = newDist
                h = heuristic(nx, ny, gx, gy)
                heapq.heappush(pq, (newDist + h, newDist, nx, ny))
    return distance, counter


In [25]:
source = (0, 0)
goal = (199, 199)

startTime=time.perf_counter()
dist_astar, expanded_astar = astar(source, goal, grid)
endTime=time.perf_counter()
astarTime=endTime-startTime
print("A* distance:", dist_astar[goal[0]][goal[1]])
print("A* expanded nodes:", expanded_astar)
print("A* time :", astarTime)

A* distance: 299.5878784786809
A* expanded nodes: 8452
A* time : 0.042625783999937994


**Dijkstras:-** <br>
Time:- 0.099 <br>
Path cost:- 299.5878 <br>
Expanded Nodes:-34527<br>

 **A***  :- <br>
Time:- 0.045 <br>
Path cost:- 299.5878 <br>
Expanded Nodes:-8452<br>

**NOTE** The distance is computed by assuming that we are allowed to travell through the corners of obstacles diagonally 

The choosen heuristic does not overestimate remaining cost because the heuristic finds the shortest possible path in an obstacle free grid when movement is allowed in all 8 directions which is an ideal case of the given problem (since the given problem also has obstacles) 